In [2]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import numpy as np
import sqlite3
import math

Syftet med den här är att mäta avståndet mellan huvudgruppernas medelpunkter.

In [107]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs limit 4000')
for row in rows:
        result.append(row)

db_contents = np.array(result)
print (db_contents)

conn.close()

[['Talg nöt' '1' 656.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86 None]
 ['Späck gris' '2' 763.0 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ['Ister gris' '3' 884.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ..., 
 ['Havredryck oberikad' '5964' 40.6 ...,
  'Vegetabiliska produkter och mjölkersättning' 46 None]
 ['Kryddblandning taco' '5973' 372.6 ..., 'Kryddor' 112 None]
 ['Tortilla wrap' '5974' 313.7 ..., 'Mjukt bröd' 12 'train']]


In [143]:
columns = np.array([ 2,  4,  5,  6])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]
# Kolumner som inte predicerar så mycket: [22, 24, 29, 30, 31, 41, 45]

In [109]:
dataset = db_contents[:,columns].astype(float)
#means = np.mean(dataset, axis=0)

label_types = np.array(list(set(db_contents.T[60]))).reshape(-1, 1) 

#Skapa 
label_types=np.hstack((label_types,np.empty([len(label_types),len(columns)])))

In [110]:
# Fyll matrisen label_types med medelvärdena i alla kolumner
for row in label_types:
    a = np.where(db_contents[:,60]==row[0]) # a blir index för alla livsmedel som tillhör den aktuella huvudgruppen
    x=np.mean(dataset[a], axis=0) # blir medelvärdena kolumn för kolumn för den aktuella huvudgruppens livsmedel
    row[1:]=x # Sätt medelvärdena på plats i den stora matrisen som är label_types/huvudgrupper
    print(row)

['' '216.54285714285717' '11.528571428571428' '9.6' '14.385714285714286']
['Salt' '32.075' '5.725' '0.125' '1.9']
['Vin' '92.76000000000002' '7.12' '0.0' '0.01']
['Kakaoprodukter' '398.1' '49.15' '12.2' '14.28']
['Frukt färsk fryst' '58.36904761904763' '11.59928571428571'
 '0.36595238095238103' '0.7402380952380951']
['Dessertost' '329.0625' '1.1375' '27.34375' '20.338749999999997']
['Godis ej choklad' '377.37272727272733' '85.44545454545455'
 '1.9363636363636365' '3.1681818181818175']
['Tuggummi' '365.3' '89.4' '0.0' '0.5']
['Sockerfritt godis' '401.8' '98.9' '0.0' '0.0']
['Hård matfettsblandning' '502.23076923076934' '1.669230769230769'
 '55.82051282051282' '0.5002564102564103']
['Jäst bakpulver' '214.5' '34.86666666666667' '0.6666666666666666'
 '16.400000000000002']
['Mjölkdryck chokladdryck milkshake smothie m yoghurt' '74.94000000000001'
 '10.41' '2.112' '3.3879999999999995']
['Te' '78.3' '15.7' '0.1' '2.925']
['Välling' '214.5833333333333' '28.617500000000003' '6.406666666666666'


In [137]:
threshold = 0.9999

In [138]:
cos_sim = cosine_similarity(label_types[:,1:], Y=None)

In [141]:
similars = np.where(cos_sim[4]>threshold)

In [142]:
label_types[similars]

array([['Frukt färsk fryst', '58.36904761904763', '11.59928571428571',
        '0.36595238095238103', '0.7402380952380951'],
       ['Pasta', '225.41904761904763', '45.50761904761905',
        '1.3538095238095242', '5.967142857142856'],
       ['Kryddor', '372.6', '72.6', '3.55', '5.63'],
       ['Potatis', '91.4', '18.006451612903227', '0.6806451612903234',
        '1.9664516129032261'],
       ['Bär färska frysta', '59.10416666666668', '11.01625',
        '0.4341666666666666', '0.8183333333333332']],
      dtype='<U73')

In [90]:
cos_sim_avg = cosine_similarity(label_types_avg, Y=None)

In [92]:
np.mean(cos_sim_avg[0])

0.98279367702164222

In [101]:
np.argsort (cos_sim[1])[-2]

79

In [100]:
np.argsort (cos_sim_avg[1])

array([101,  65, 115,  41,  92,  70,  24,  14,  32,   9, 116,  38,  36,
        25,  39,  89,  60,  86,   5,  40,  46,  98,  30,  91,  56,  42,
        47,  96,  16,  93, 106,  58,  74,  43,  66,  61,  34,  27,  26,
        68,   0,  18,  84, 105,  23,  37,  63,  17,  15, 114,  49, 110,
        75,   8,  53, 104,  76,  20,  33,  28, 100,   7,  48,  87,  64,
       111,  54,  94,  83,  59,  73,  82,  35,   2,  51, 102, 117,  19,
         6,  22,  57,   3,  31,  85, 103,  95,  62,  71,  21,  90,  77,
       107,  13,   4,  45,  11, 108,  80,  29, 112,  55,  88,  81,  72,
       118,  97,  44,  78,  69,  99,  12,  52, 109,  67, 113,  10,  50,
        79,   1])

In [ ]:
np.where

In [53]:
similar = label_types[np.argsort (cos_sim)][::-1] #[::-1] vänder på listan https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order

In [103]:
similar.shape

(119, 119, 5)

In [99]:
print(np.mean(label_types_avg, axis=1, keepdims=True))

[[  5.32907052e-15]
 [  4.44089210e-16]
 [ -2.66453526e-15]
 [  1.06581410e-14]
 [  0.00000000e+00]
 [  3.55271368e-15]
 [ -3.55271368e-15]
 [ -7.10542736e-15]
 [ -7.10542736e-15]
 [  1.42108547e-14]
 [ -3.55271368e-15]
 [ -8.88178420e-16]
 [  2.66453526e-15]
 [  7.10542736e-15]
 [ -1.42108547e-14]
 [ -3.55271368e-15]
 [ -7.10542736e-15]
 [ -4.44089210e-16]
 [  0.00000000e+00]
 [  4.44089210e-16]
 [ -3.55271368e-15]
 [  0.00000000e+00]
 [ -1.77635684e-15]
 [  8.88178420e-16]
 [  2.84217094e-14]
 [  1.77635684e-15]
 [  0.00000000e+00]
 [ -3.55271368e-15]
 [ -1.33226763e-15]
 [  7.10542736e-15]
 [  3.55271368e-15]
 [  1.77635684e-14]
 [ -2.84217094e-14]
 [ -8.88178420e-16]
 [  7.10542736e-15]
 [ -1.06581410e-14]
 [  8.88178420e-16]
 [  8.88178420e-16]
 [  3.55271368e-15]
 [  3.55271368e-15]
 [ -3.55271368e-15]
 [ -1.77635684e-15]
 [ -1.77635684e-15]
 [  1.77635684e-15]
 [  1.33226763e-15]
 [ -2.66453526e-15]
 [  0.00000000e+00]
 [ -1.24344979e-14]
 [ -3.55271368e-15]
 [ -4.44089210e-16]
